In [1]:
import numpy as np
from tqdm import tqdm

np.random.seed(102191)

#calcula cuantos encestes logra un jugador con indice de enceste prob
#haciendo qyt tiros libres

def ftirar(prob, qty):
    return sum(np.random.rand(qty) < prob)

def create_peloton(num_values):
    return np.linspace(0.501, 0.600, num_values)

def montecarlo(jugadores,mejor,cantidad_tiros,simulaciones):
    peloton = create_peloton(jugadores)
    total_jugadores = np.append(mejor, peloton)
    vec_ftirar = np.vectorize(ftirar)

    primero_ganador = 0

    for i in tqdm(range(simulaciones)): 
        vaciertos = vec_ftirar(total_jugadores, cantidad_tiros)
        mejor_jug = np.argmax(vaciertos)
        if mejor_jug == 0:
            primero_ganador += 1

    print(f'el modelo elije al real mejor un {100*round(primero_ganador/simulaciones,4)}% de las veces')
    return primero_ganador/simulaciones

In [2]:
mejor = 0.7
simulaciones = 10000

### Cazatalentos 1: Llegué a la localidad habían 60 adolescentes en el gimnasio, hice tirar a cada uno 100 tiros, solo uno obtuvo 80 aciertos, luego un par 79, otro par 78, y así descendían.

In [13]:
lista_def = []

In [3]:
jugadores = 60
cantidad_tiros = 100

res_1 = montecarlo(jugadores,mejor,cantidad_tiros,simulaciones)
lista_def.append((res_1*80, 'caz_1'))

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:09<00:00, 1029.88it/s]

el modelo elije al real mejor un 68.78% de las veces


In [7]:
# cazatalentos 2
jugadores = 200
cantidad_tiros = 100

res_2 = montecarlo(jugadores,mejor,cantidad_tiros,simulaciones)
lista_def.append((res_2*80, 'caz_2'))

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:29<00:00, 335.60it/s]

el modelo elije al real mejor un 52.0% de las veces


In [20]:
# cazatalentos 3
jugadores = 2
cantidad_tiros = 100

res_3 = montecarlo(jugadores,mejor,cantidad_tiros,simulaciones)
lista_def.append((res_3*80, 'caz_3'))

100%|█████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 12192.38it/s]

el modelo elije al real mejor un 93.78% de las veces


In [23]:
# cazatalentos 4
jugadores = 50
cantidad_tiros = 10

res_4 = montecarlo(jugadores,mejor,cantidad_tiros,simulaciones)
lista_def.append((res_4*90, 'caz_4'))

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:04<00:00, 2472.68it/s]

el modelo elije al real mejor un 20.119999999999997% de las veces


In [24]:
# cazatalentos 5
jugadores = 1
cantidad_tiros = 100

res_5 = montecarlo(jugadores,mejor,cantidad_tiros,simulaciones)
lista_def.append((res_5*80, 'caz_5'))

100%|█████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 15160.31it/s]

el modelo elije al real mejor un 99.92999999999999% de las veces


In [25]:
def cazatalentos_ronda1(jugadores, num_seleccionados, cantidad_tiros):
    peloton = create_peloton(jugadores)
    vec_ftirar = np.vectorize(ftirar)
    vaciertos = vec_ftirar(peloton, cantidad_tiros)
    mejores_indices = np.argpartition(vaciertos, -num_seleccionados)[-num_seleccionados:]
    mejores = peloton[mejores_indices]
    return mejores

def cazatalentos_6(jugadores, num_seleccionados, cantidad_tiros_ronda1, mejor, cantidad_tiros_ronda2, simulaciones):
    primero_ganador = 0
    
    for i in tqdm(range(simulaciones)):
        # Ronda 1: seleccionamos los mejores jugadores
        mejores_ronda1 = cazatalentos_ronda1(jugadores, num_seleccionados, cantidad_tiros_ronda1)
        
        # Ronda 2: aplicamos el proceso de Monte Carlo con los mejores jugadores de la Ronda 1
        total_jugadores = np.append(mejor, mejores_ronda1)
        vec_ftirar = np.vectorize(ftirar)
        vaciertos = vec_ftirar(total_jugadores, cantidad_tiros_ronda2)
        mejor_ronda2 = np.argmax(vaciertos)
        
        if mejor_ronda2 == 0:
            primero_ganador += 1
    
    print(f'el modelo elije al real mejor un {100*round(primero_ganador/simulaciones,4)}% de las veces')
    return primero_ganador/simulaciones

In [26]:
jugadores = 100
num_seleccionados = 5
cantidad_tiros_ronda1 = 50
cantidad_tiros_ronda2 = 100

res_6 = cazatalentos_6(jugadores, num_seleccionados, cantidad_tiros_ronda1, mejor, cantidad_tiros_ronda2, simulaciones)
lista_def.append((res_6*80, 'caz_6'))

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:12<00:00, 771.69it/s]

el modelo elije al real mejor un 88.37% de las veces


In [27]:
def cazatalentos_7(jugadores, num_seleccionados, mejor, cantidad_tiros_ronda2, simulaciones):
    primero_ganador = 0

    for i in tqdm(range(simulaciones)):
        # Ronda 1: eliminamos jugadores si fallan un tiro libre
        peloton_ronda1 = create_peloton(jugadores)
        jugadores_activos = np.copy(peloton_ronda1)
        while len(jugadores_activos) > num_seleccionados:
            tiros = np.random.rand(len(jugadores_activos))
            jugadores_activos = jugadores_activos[tiros < jugadores_activos]

        # Ronda 2: aplicamos el proceso de Monte Carlo con los mejores jugadores de la Ronda 1
        total_jugadores = np.append(mejor, jugadores_activos)
        vec_ftirar_ronda2 = np.vectorize(ftirar)
        vaciertos_ronda2 = vec_ftirar_ronda2(total_jugadores, cantidad_tiros_ronda2)
        mejor_ronda2 = np.argmax(vaciertos_ronda2)

        if mejor_ronda2 == 0:
            primero_ganador += 1

    print(f'el modelo elije al real mejor un {100*round(primero_ganador/simulaciones,4)}% de las veces')
    return primero_ganador/simulaciones

In [28]:
jugadores = 100
num_seleccionados = 5
cantidad_tiros_ronda2 = 100

res_7 = cazatalentos_7(jugadores, num_seleccionados, mejor, cantidad_tiros_ronda2, simulaciones)
lista_def.append((res_7*80, 'caz_7'))

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 5997.45it/s]

el modelo elije al real mejor un 93.35% de las veces


In [29]:
def cazatalentos_8(jugadores, jugadores_con_datos, mejor, cantidad_tiros_ronda1, simulaciones):
    primero_ganador = 0

    for i in tqdm(range(simulaciones)):
        # Primero, asumimos que el verdadero mejor jugador está entre los jugadores con datos
        peloton_con_datos = create_peloton(jugadores_con_datos)
        peloton_sin_datos = create_peloton(jugadores - jugadores_con_datos)
        peloton = np.concatenate((peloton_con_datos, peloton_sin_datos))
        
        total_jugadores = np.append(mejor, peloton)
        vec_ftirar_ronda1 = np.vectorize(ftirar)
        vaciertos_ronda1 = vec_ftirar_ronda1(total_jugadores, cantidad_tiros_ronda1)
        mejor_ronda1 = np.argmax(vaciertos_ronda1)

        if mejor_ronda1 == 0:
            primero_ganador += 1

    print(f'el modelo elije al real mejor un {100*round(primero_ganador/simulaciones,4)}% de las veces')
    return primero_ganador/simulaciones

In [34]:
jugadores = 100
jugadores_con_datos = 30
cantidad_tiros_ronda1 = 100

res_8 = cazatalentos_8(jugadores, jugadores_con_datos, mejor, cantidad_tiros_ronda1, simulaciones)
lista_def.append((res_8*85, 'caz_8'))

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:16<00:00, 604.40it/s]

el modelo elije al real mejor un 61.01% de las veces


In [35]:
def cazatalentos_9(jugadores, cantidad_tiros_observacion, mejores_seleccionados, mejor, cantidad_tiros_ronda2, simulaciones):
    primero_ganador = 0
    
    for i in tqdm(range(simulaciones)):
        # Primera ronda: observación de los jugadores sin tomar en cuenta los encestes
        peloton = create_peloton(jugadores)
        
        # Segunda ronda: aplicamos el proceso de Monte Carlo con los mejores jugadores seleccionados en la observación
        mejores_jugadores = np.random.choice(peloton, size=mejores_seleccionados, replace=False)
        total_jugadores = np.append(mejor, mejores_jugadores)
        vec_ftirar_ronda2 = np.vectorize(ftirar)
        vaciertos_ronda2 = vec_ftirar_ronda2(total_jugadores, cantidad_tiros_ronda2)
        mejor_ronda2 = np.argmax(vaciertos_ronda2)
        
        if mejor_ronda2 == 0:
            primero_ganador += 1

    print(f'el modelo elije al real mejor un {100*round(primero_ganador/simulaciones,4)}% de las veces')
    return primero_ganador/simulaciones

In [36]:
jugadores = 100
cantidad_tiros_observacion = 5
mejores_seleccionados = 2
cantidad_tiros_ronda2 = 100

res_9 = cazatalentos_9(jugadores, cantidad_tiros_observacion, mejores_seleccionados, mejor, cantidad_tiros_ronda2, simulaciones)
lista_def.append((res_9*81, 'caz_9'))

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 6881.36it/s]

el modelo elije al real mejor un 96.61% de las veces


In [38]:
sorted(lista_def)

[(18.108, 'caz_4'),
 (41.6, 'caz_2'),
 (51.8585, 'caz_8'),
 (55.024, 'caz_1'),
 (70.696, 'caz_6'),
 (74.68, 'caz_7'),
 (75.024, 'caz_3'),
 (78.2541, 'caz_9'),
 (79.944, 'caz_5')]